### YouTube Statistics - Capstone Project 
#### Exploring the Relationship between Engagement Metrics, Keywords, and Comment Sentiment

#### BY: Daphney Oliveira

In [1]:
#importing the libraries
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import jupyterthemes 

In [2]:
#Reading and looking at the data in the comments file

comments_df = pd.read_csv("comments.csv") 
comments_df.info()
comments_df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18409 entries, 0 to 18408
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  18409 non-null  int64  
 1   Video ID    18409 non-null  object 
 2   Comment     18408 non-null  object 
 3   Likes       18409 non-null  float64
 4   Sentiment   18409 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 719.2+ KB


,Unnamed: 0,Likes,Sentiment
count,18409.000000,18409.000000,18409.000000
mean,9204.000000,1040.019447,1.493998
std,5314.364888,10651.366148,0.709928
min,0.000000,0.000000,0.000000
25%,4602.000000,5.000000,1.000000
50%,9204.000000,29.000000,2.000000
75%,13806.000000,190.000000,2.000000
max,18408.000000,891372.000000,2.000000


In [3]:
#Reading and looking at the data in the video-stats file

videostats_df = pd.read_csv("videos-stats.csv") 
videostats_df.info()
videostats_df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1881 entries, 0 to 1880
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    1881 non-null   int64  
 1   Title         1881 non-null   object 
 2   Video ID      1881 non-null   object 
 3   Published At  1881 non-null   object 
 4   Keyword       1881 non-null   object 
 5   Likes         1879 non-null   float64
 6   Comments      1879 non-null   float64
 7   Views         1879 non-null   float64
dtypes: float64(3), int64(1), object(4)
memory usage: 117.7+ KB


,Unnamed: 0,Likes,Comments,Views
count,1881.000000,1.879000e+03,1879.000000,1.879000e+03
mean,940.000000,1.700610e+05,7863.331559,1.161292e+07
std,543.142247,7.962293e+05,37879.964926,1.084450e+08
min,0.000000,-1.000000e+00,-1.000000,2.500000e+01
25%,470.000000,2.672500e+03,199.000000,8.451500e+04
50%,940.000000,1.478700e+04,814.000000,5.917210e+05
75%,1410.000000,6.090600e+04,3377.500000,2.804978e+06
max,1880.000000,1.644556e+07,732818.000000,4.034122e+09


In [4]:
#Determining the dimensions of both datasets

print("Comments Dataset is of ", comments_df.ndim, " dimensions.")
print("Video-Stats Dataset is of ", videostats_df.ndim, " dimensions.")

Comments Dataset is of  2  dimensions.
Video-Stats Dataset is of  2  dimensions.


In [5]:
#Determining the shape of both datasets

print("Comments Dataset has ", comments_df.shape[0], " rows","and ", comments_df.shape[1], "columns.")
print("Video-Stats Dataset has ", videostats_df.shape[0], " rows","and ", videostats_df.shape[1], "columns.")

Comments Dataset has  18409  rows and  5 columns.
Video-Stats Dataset has  1881  rows and  8 columns.


In [6]:
#Looking at the datatypes in the comments file

print("Comments - Data Types: ")
print(comments_df.dtypes)

Comments - Data Types: 
Unnamed: 0      int64
Video ID       object
Comment        object
Likes         float64
Sentiment     float64
dtype: object


In [7]:
#Looking at the datatypes in the video-stats file

print("Video-Stats - Data Types: ")
print(videostats_df.dtypes)

Video-Stats - Data Types: 
Unnamed: 0        int64
Title            object
Video ID         object
Published At     object
Keyword          object
Likes           float64
Comments        float64
Views           float64
dtype: object


### Data Cleaning
#### Dropping the unnamed column, renaming columns prior to joining tables, and removing punctuation and stop words

#### I'll begin by cleaning the comments dataframe:

In [11]:
#dropped the unnamed column from the comments dataframe

comm = comments_df.drop(columns='Unnamed: 0')

In [9]:
comm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18409 entries, 0 to 18408
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Video ID   18409 non-null  object 
 1   Comment    18408 non-null  object 
 2   Likes      18409 non-null  float64
 3   Sentiment  18409 non-null  float64
dtypes: float64(2), object(2)
memory usage: 575.4+ KB


In [36]:
#renaming the columns before joining this df with the video-stats df

#print(comm.columns)

#comm.rename(columns = {'Video ID': 'video_id', 'Comment': 'comment', 'Likes': 'comment_likes', 'Sentiment': 'comment_sentiment'}, inplace = True)
comm.columns

Index(['video_id', 'comment', 'comment_likes', 'comment_sentiment'], dtype='object')

#### Now let's clean the video-stats dataframe:

In [13]:
#dropped the unnamed column from the stats dataframe

videostats = videostats_df.drop(columns='Unnamed: 0')
videostats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1881 entries, 0 to 1880
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Title         1881 non-null   object 
 1   Video ID      1881 non-null   object 
 2   Published At  1881 non-null   object 
 3   Keyword       1881 non-null   object 
 4   Likes         1879 non-null   float64
 5   Comments      1879 non-null   float64
 6   Views         1879 non-null   float64
dtypes: float64(3), object(4)
memory usage: 103.0+ KB


In [14]:
#renaming the columns before joing this df with the comments df

print(videostats.columns)
videostats.rename(columns = {'Title': 'title', 'Video ID': 'video_id', 'Published At': 'published_at', 'Keyword': 'keyword', 'Likes': 'video_likes', 'Comments': 'video_comments', 'Views': 'video_views'}, inplace = True)
videostats.columns

Index(['Title', 'Video ID', 'Published At', 'Keyword', 'Likes', 'Comments',
       'Views'],
      dtype='object')


Index(['title', 'video_id', 'published_at', 'keyword', 'video_likes',
       'video_comments', 'video_views'],
      dtype='object')

## Manipulating Data
##### Time to organize and arrange the dataframes. Prior to combining both the comments and video stats dataframes, I will remove duplicates and see if there are any nulls.

In [38]:
comm.isnull().sum()

video_id             0
comment              1
comment_likes        0
comment_sentiment    0
dtype: int64

In [39]:
comments = comm.dropna()

In [40]:
comments.isnull().sum()

video_id             0
comment              0
comment_likes        0
comment_sentiment    0
dtype: int64

In [41]:
videostats.isnull().sum()

title             0
video_id          0
published_at      0
keyword           0
video_likes       2
video_comments    2
video_views       2
dtype: int64

In [18]:
stats = videostats.dropna()

In [19]:
stats.isnull().sum()

title             0
video_id          0
published_at      0
keyword           0
video_likes       0
video_comments    0
video_views       0
dtype: int64

In [24]:
# There are 2 nulls in the videostats df in 3 columns and 1 null in the comm df in 1 column. I removed all nulls.

In [42]:
#combining both stats and comments datasets 

videostats_comments = pd.merge(stats, comments, on=['video_id'])

In [27]:
# Now let's check if there any duplicates and where they're located
videostats.duplicated().sum()
comments.duplicated().sum()

148

In [43]:
videostats_comments.duplicated().sum()

258

In [45]:
#removing the duplicates from the merged dataset
dup_free = videostats_comments.drop_duplicates(subset=['comment'], keep='first', inplace=False, ignore_index=False) #removing the dups and creating new DF in case I can keep the dups
dup_free.duplicated().sum()

0

In [46]:
videostats_comments.describe(include=['object']) #before dup removal

,title,video_id,published_at,keyword,comment
count,18624,18624,18624,18624,18624
unique,1853,1867,756,41,17974
top,20 Minecraft Block Facts You Maybe Didn&#39;t ...,ZgeorpjGJC0,2022-08-24,reaction,One of the best thing about Dude is that he\r\...
freq,40,40,2841,519,20


In [47]:
dup_free.describe(include=['object']) #after dup removal

,title,video_id,published_at,keyword,comment
count,17974,17974,17974,17974,17974
unique,1852,1866,756,41,17974
top,Tiësto - The Business (Lyrics),wAZZ-UWGVHI,2022-08-24,tutorial,Let's not forget that Apple Pay in 2014 requir...
freq,30,10,2613,500,1


In [48]:
#exporting a new csv and xlsx without the dups

dup_free.to_csv('vstatsc_dupfree.csv', index=False)
dup_free.to_excel('youtubestats_2.xlsx', index=False)

## Exploratory Data Analysis
#### Keep Note:
##### Dirty datasets:
    * comm
    * videostats

##### Clean datasets: 
    * comments
    * stats

##### Clean datasets merged:
    * videostats_comments
    
    
##### Clean datasets merged & duplicate free:
    * dup_free

In [51]:
#basic statistics

dup_free.describe().apply(lambda s: s.apply(lambda x: format(x, 'g')))

,video_likes,video_comments,video_views,comment_likes,comment_sentiment
count,17974,17974,17974,17974,17974
mean,168822,7862.61,1.18297e+07,992.542,1.48971
std,787759,37786.1,1.1057e+08,9582.96,0.71287
min,-1,-1,25,0,0
25%,2945,222,95369,5,1
50%,15576,864,609802,30,2
75%,61448,3469,2.85376e+06,195,2
max,1.64456e+07,732818,4.03412e+09,891372,2


In [52]:
dup_free.describe(include=['object'])

,title,video_id,published_at,keyword,comment
count,17974,17974,17974,17974,17974
unique,1852,1866,756,41,17974
top,Tiësto - The Business (Lyrics),wAZZ-UWGVHI,2022-08-24,tutorial,Let's not forget that Apple Pay in 2014 requir...
freq,30,10,2613,500,1


In [ ]:
# After cleaning the datasets and looking at the basic statistics, I used excel pivot tables to perform exploratory data analysis. Then I used tableau to create my visuals.